In [1]:
from collections import Counter
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor

In [4]:
train = pd.read_csv('data/train.csv')

## Data
- 3498개 화합물 정보
- id: 샘플 고유 id
- SMILES: 화합물의 분자 구조
- 화합물의 물성 관련 정보
- MLM(Mouse Liver Microsome) 포함
- HLM(Human Liver Microsome) 포함
- HLM 및 MLM은 간 및 마우스의 간 대사효소와 화합물을 30분 동안 반응시킨 후, 
- 대사되지 않고 남아있는 화합물의 양을(%) LC-MS/MS로 측정함으로써 화합물의 간 대사효소에 대한 안정성을 평가한 데이터
- smiles: https://process-mining.tistory.com/158

In [7]:
# seed
seed = 42

# define rmse
rmse = lambda x, y: np.mean((x - y) ** 2) ** 0.5 

# features and targets
features = ["AlogP", "Molecular_Weight", "Num_H_Acceptors", "Num_H_Donors", "Num_RotatableBonds", "LogD", "Molecular_PolarSurfaceArea"]
mlm_target = "MLM"
hlm_target = "HLM"

# load data
df = pd.read_csv("data/train.csv")
df["AlogP"] = np.where(pd.isna(df["AlogP"]), df["LogD"], df["AlogP"])

In [14]:
df.corr()

,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea
MLM,1.000000,0.706725,-0.330556,-0.081239,0.164510,0.210836,-0.092563,-0.350146,0.184850
HLM,0.706725,1.000000,-0.346654,-0.175117,0.092313,0.176549,-0.132263,-0.357456,0.094323
AlogP,-0.330556,-0.346654,1.000000,0.389908,-0.284302,-0.172566,0.112179,0.957647,-0.298620
Molecular_Weight,-0.081239,-0.175117,0.389908,1.000000,0.471814,0.116186,0.583711,0.369462,0.439114
Num_H_Acceptors,0.164510,0.092313,-0.284302,0.471814,1.000000,0.208433,0.474012,-0.305506,0.714315
Num_H_Donors,0.210836,0.176549,-0.172566,0.116186,0.208433,1.000000,0.176871,-0.212082,0.474614
Num_RotatableBonds,-0.092563,-0.132263,0.112179,0.583711,0.474012,0.176871,1.000000,0.071659,0.371574
LogD,-0.350146,-0.357456,0.957647,0.369462,-0.305506,-0.212082,0.071659,1.000000,-0.294670
Molecular_PolarSurfaceArea,0.184850,0.094323,-0.298620,0.439114,0.714315,0.474614,0.371574,-0.294670,1.000000


In [23]:
df.groupby('SMILES').size().sort_values()

SMILES
Brc1c[nH]nc1-c1nc2c3ccccc3ncn2n1                         1
Cc1cccc(OCCn2cnc3c(cnn3-c3cccc(C(F)(F)F)c3)c2=O)c1       1
Cc1cccc(S(=O)(=O)Nc2nc3ncc(Br)nc3s2)c1                   1
Cc1cccc2c(=O)n(CC(=O)NCC3CC4c5ccccc5C3c3ccccc34)cnc12    1
Cc1cccc2c(O)c3nc(C)n(-c4ccccc4)c3nc12                    1
                                                        ..
N#Cc1nccnc1OC1CCN(C(=O)N2CCNC2=O)C1                      2
N#Cc1nccnc1OC1CCN(C(=O)C2CC(=O)N(C3CCOCC3)C2)C1          2
COc1ccc([C@H]2Oc3cc(OC)cc(OC)c3C[C@H]2O)cc1              2
CCCCC/N=c1\n(C)c(=O)nc2sccn12                            2
COc1ccc(C2=NOC(c3ccsc3)C2)cc1                            2
Length: 3471, dtype: int64

In [27]:
df[df['SMILES'] == 'N#Cc1nccnc1OC1CCN(C(=O)N2CCNC2=O)C1']

,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea
2159,TRAIN_2159,N#Cc1nccnc1OC1CCN(C(=O)N2CCNC2=O)C1,113.210,99.000,-0.533,302.289,6,1,2,-0.533,111.44
2332,TRAIN_2332,N#Cc1nccnc1OC1CCN(C(=O)N2CCNC2=O)C1,0.007,0.038,-0.533,302.289,6,1,2,-0.533,111.44


In [20]:
df.sort_values('SMILES')

,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea
3302,TRAIN_3302,Brc1c[nH]nc1-c1nc2c3ccccc3ncn2n1,99.000,99.000,2.575,315.128,4,1,1,2.702,71.75
623,TRAIN_0623,Brc1cc(-c2noc([C@H]3CCCO3)n2)c2ncnn2c1,3.780,35.920,1.357,336.144,5,0,2,1.357,78.34
141,TRAIN_0141,Brc1ccc(-c2c[nH]c(CNc3nncn3-c3ccccc3)n2)cc1,8.989,50.893,3.686,395.256,3,2,5,3.725,71.42
305,TRAIN_0305,Brc1ccc(-c2n[nH]c(-c3ccccc3)c2-c2ncc[nH]2)cc1,39.870,85.910,4.591,365.227,2,2,3,4.871,57.36
3294,TRAIN_3294,Brc1ccc(C2CC(c3ccc(Br)cc3)n3ncnc3N2)cc1,42.647,59.504,4.748,434.128,2,1,2,4.741,42.74
...,...,...,...,...,...,...,...,...,...,...,...
2409,TRAIN_2409,c1cnc2c(c1)OC(Cn1nnc3ncc(-c4ccc(OCC5CCNCC5)cc4...,86.566,91.896,2.494,458.516,8,2,6,1.587,111.90
3439,TRAIN_3439,c1csc(COc2ccc(CNn3cnnc3)cc2)c1,56.030,84.800,1.798,286.352,4,1,6,1.798,80.21
1717,TRAIN_1717,c1nc(N2CCCC2)sc1CN1CC[C@H]2OCc3cnnn3[C@H]2C1,80.400,99.050,1.072,346.450,5,0,3,0.715,87.55
2197,TRAIN_2197,c1nc(N2CCN(C3CCCCC3)CC2)c2nc[nH]c2n1,25.370,97.100,2.282,286.375,4,1,2,0.452,60.94


In [19]:
df['SMILES'].sample(5)

2477    CC1(C)C[C@H](O)[C@@H](n2/c(=N/C#N)[nH]c3ccccc3...
633     O=C(NC1CCCC1)N1Cc2c(sc3c2CCCC3)-n2cccc2C1c1ccc...
3272                          Brc1cccc(-c2cnc3n2CCCCC3)c1
1851                    Cc1cc(C)n(CCCN(C)CC2(O)CNCCOC2)n1
1718              CC1CCC(N(Cc2nnc(C3CCC3)n2C)CC2CCCO2)CC1
Name: SMILES, dtype: object

In [13]:
# train
scores = []
reg_mlms = []
reg_hlms = []
kf = KFold(n_splits = 10, random_state = seed, shuffle = True)
for i, (train_index, valid_index) in enumerate(kf.split(df)):
    df_train = df.iloc[train_index]
    df_valid = df.iloc[valid_index]

    x_train = df_train[features].values
    y_mlm_train = df_train[mlm_target].values
    y_hlm_train = df_train[hlm_target].values

    x_valid = df_valid[features].values
    y_mlm_valid = df_valid[mlm_target].values
    y_hlm_valid = df_valid[hlm_target].values

    reg_mlm = RandomForestRegressor(random_state = seed)
    reg_mlm.fit(x_train, y_mlm_train)
    p_mlm = reg_mlm.predict(x_valid)

    reg_hlm = RandomForestRegressor(random_state = seed)
    reg_hlm.fit(x_train, y_hlm_train)
    p_hlm = reg_hlm.predict(x_valid)

    score = 0.5 * rmse(y_mlm_valid, p_mlm) + 0.5 * rmse(y_hlm_valid, p_hlm)

    reg_mlms.append(reg_mlm)
    reg_hlms.append(reg_hlm)
    scores.append(score)
    print(f"Fold {i+1:2d}: {score:.5f}")

print(f"CV score: {np.mean(scores):.5f}")

Fold  1: 34.16658
Fold  2: 34.03305
Fold  3: 33.21484
Fold  4: 34.63931
Fold  5: 32.44617
Fold  6: 33.18904
Fold  7: 31.83381
Fold  8: 31.91317
Fold  9: 31.68066
Fold 10: 33.10630
CV score: 33.02229


In [ ]:
# load data
df = pd.read_csv("data/test.csv")
df["AlogP"] = np.where(pd.isna(df["AlogP"]), df["LogD"], df["AlogP"])

# predict
df_submission = pd.read_csv("data/sample_submission.csv")
df_submission["MLM"] = np.mean([reg_mlm.predict(df[features].values) for reg_mlm in reg_mlms], axis = 0)
df_submission["HLM"] = np.mean([reg_hlm.predict(df[features].values) for reg_hlm in reg_hlms], axis = 0)
df_submission.to_csv("submission.csv", index = False, encoding = "utf-8-sig")

## SMILES

In [16]:
import numpy as np

# define SMILES characters ----------------------------------------------------
SMILES_CHARS = [' ',
                '#', '%', '(', ')', '+', '-', '.', '/',
                '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                '=', '@',
                'A', 'B', 'C', 'F', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P',
                'R', 'S', 'T', 'V', 'X', 'Z',
                '[', '\\', ']',
                'a', 'b', 'c', 'e', 'g', 'i', 'l', 'n', 'o', 'p', 'r', 's',
                't', 'u']
                
# define encoder and decoder --------------------------------------------------
smi2index = dict( (c,i) for i,c in enumerate( SMILES_CHARS ) )
index2smi = dict( (i,c) for i,c in enumerate( SMILES_CHARS ) )

def smiles_encoder( smiles, maxlen=120 ):
    X = np.zeros( ( maxlen, len( SMILES_CHARS ) ) )
    for i, c in enumerate( smiles ):
        X[i, smi2index[c] ] = 1
    return X

def smiles_decoder( X ):
    smi = ''
    X = X.argmax( axis=-1 )
    for i in X:
        smi += index2smi[ i ]
    return smi

# get a taste of caffeine -----------------------------------------------------
caffeine_smiles = 'CN1C=NC2=C1C(=O)N(C(=O)N2C)C'

caffeine_encoding = smiles_encoder(caffeine_smiles)

print(caffeine_encoding.shape) # (120, 56)

(120, 56)


In [17]:
caffeine_encoding

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])